# Remove, Replace, Compare (RRC) Algorithm

Adapted from [O. Guyon](http://www.naoj.org/staff/guyon/09allskysurvey.web/56photometry.web/content.html) by W. Gee

An image sequence is a group of images taken by one camera corresponding to one observation. An observation is defined by mount slews such that there is no slewing of the mount between images in a sequence (by definition, a meridian flip to the same target signals the beginning of a new observation).

For each image in the sequence, a postage stamp is created for each point source contained within the PANOPTES Image Catalog (PIC).

One individual image can contain up to 100,000 postage stamps depending on the stellar density of the observed field. To test for a transit, the target postage stamp is compared to all other postage stamps within the field.

### Step 1: Choose Transit Parameters

Parameters: _phase_, _period_, _duration_

The purpose of the steps below is to test if the transit matching these parameters occured. The metric used is the transit depth, computed at the end of the steps.

### Step 2: Separate Postage Stamps

Postage stamps in the sequence are separated into two groups:

* **Group A**: All postage stamps that occur during time $t$ for which a transit depth is being tested.
* **Group B**: All postage stamps outside time $t$.

### Step 3: Identify Reference PSFs

Use all the postage stamps in Group B (outside transit time $t$) to identify PSF(s) in the field which best match the target PSF.

The identification is done directly from the postage stamps (no photometry, no lightcurve) using the sum square difference between the target postage stamp and PSFs as a minimization criteria. 

For example, the following criteria may be used:

$$ 
V_{(x_r,y_r)} = \sum_{i=0}^{n-1}
    {\big[ 
        \sum_{u=-w}^{w}{ 
            \big[
            \sum_{v=-w}^{w}{
            \big( 
                B_i(x_t + u, y_t + v) - B_i(x_r + u, y_r + v)
            \big)^2 
            \big]
        }}
    \big]} 
$$

$$(1)$$

where:
* $i$ is the postage stamp index within set $B$
* $n$ is the number of postage stamps in set $B$
* $u$ is the horizontal pixel offset
* $v$ is the vertical pixel offset
* $w$ denotes half of the postage stamp width
* $(x_r,y_r)$ is the location of the star within the reference postage stamp
* $(x_t,y_t)$ is the location of the star within the target postage stamp, and
* $B_i(x + u,y + v)$ is the pixel value for pixel $(x + u,y + v)$ in postage stamp $i$ within group $B$.

This can be read as: for each postage stamp outside time $t$, find the (squared) difference between the sum of all the pixels in the target stamp and the sum of the pixels in the reference stamp. 

The value $V(x_r,y_r)$ then represents how well, within the group $B$ of postage stamps, the pixel values around pixel $(x_r,y_r)$ match the pixel values around pixel $(x_t,y_t)$ where the target star is located.

We note that $V(x_t,y_t)=0$, and that a small value for $V(x_r,y_r)$ indicates that a star which behaves similarly to the target star is located on pixel $(x_r,y_r)$. Equation (1) may be refined to include flux scaling (allow for comparison of stars of difference brightnesses), and weighting factors that take into account noise properties (readout noise, photon noise) or other known limitations.

### Step 4: Construct Reference PSF

The value(s) of $(x,y)$ for which $V(x,y)$ is smallest is(are) used to build this sequence. 

For example, if the smallest values of $V(x,y)$ (other than the trivial $x=x_t, y=y_t$ solution) are obtained for $(x,y)=(x_{\rm{min}_1}, y_{\rm{min}_1})$ and $(x,y)=(x_{\rm{min}_2}, y_{\rm{min}_2})$, then:

$$
    \mathrm{RefPSF}_j(u,v) = 
        \alpha \; A_j(x_{\rm{min}_1} + u,y_{\rm{min}_1} + v) + \beta \; A_j(x_{\rm{min}_2} + u,y_{\rm{min}_2} + v)
$$

$$(2)$$

where:

* $\alpha$ and $\beta$ are adjusted to best match the photometry of the target and reference PSF outside transit (group $B$), and 
* $j$ is the postage stamp index within group $A$ (during transit).

Note that in the reference PSF we are **removing** the values given by $(x_t, y_t)$ from the $A_j$ postage stamp and **replacing** them with them with combination of values given by $(x_{\rm min}, y_{\rm min})$.

### Step 5: Derive Estimated Transit Depth

Derive the depth by **comparing** the target PSF to the reference PSF within the transit postage stamp set $A$.

An estimate of the transit depth $a$ may be obtained by:

$$
(1+a) = 
    \frac{
        \displaystyle\sum_{j=0}^{m-1}
            {\big[ 
                \sum_{u=-w}^{w}{ 
                    \big[
                    \sum_{v=-w}^{w}{
                        A_j(x_t + u, y_t + v)
                    \big]
                }}
            \big]} 
    }{
        \displaystyle\sum_{j=0}^{m-1}
            {\big[ 
                \sum_{u=-w}^{w}{ 
                    \big[
                    \sum_{v=-w}^{w}{
                        \mathrm{RefPSF}_j(u, v)
                    \big]
                }}
            \big]} 
    }
$$

$$(3)$$

Where $m$ is the number of images in group $A$. This equation may be improved by optimal weighting of pixel values.